In [ ]:
import datetime
import os
import sys
import pandas as pd
import FinanceDataReader as fdr

from tabulate import tabulate
from datetime import timedelta

df_kospi = fdr.StockListing('KOSPI')
df_kosdaq = fdr.StockListing('KOSDAQ')

# 관리종목
df_AdmStock = fdr.StockListing('KRX-ADMIN')

df_krx = pd.concat([df_kospi, df_kosdaq])

# 관리종목 및 우선주 제거(5, 7, 9, K)
df_filterStk = df_krx[~df_krx['Code'].isin(df_AdmStock['Symbol'])]
df_filterStk = df_filterStk[~df_filterStk['Code'].str.endswith(('5', '7', '9', 'K'))]

df_filterStk[0:10]

In [ ]:
df_filterStk['BF_HALF_YEAR_DATE'] = None
df_filterStk['BF_HALF_YEAR_CLOSE'] = None
df_filterStk['BF_RATIo'] = 0

initDate = datetime.date(2018, 12, 28)
bfDate = initDate + timedelta(days=-180)    # 6개월 전 날짜


for index, row in df_filterStk.iterrows():
    stkCode = row['Code']
    
    try:
        # df_stockPrice = fdr.DataReader(stkCode, bfDate, initDate)
        df_stockPrice = fdr.DataReader(stkCode, bfDate, initDate) if fdr.DataReader(stkCode, bfDate, initDate).shape[0] > 0 else fdr.DataReader(stkCode)
        bfPrice = df_stockPrice.iloc[0]['Close']
        currPrice = df_stockPrice.iloc[-1]['Close']
        
        # print(f'[{stkCode}] before Date : {df_stockPrice.iloc[0].name.strftime('%Y-%m-%d')}, {bfPrice} | curr Date : {df_stockPrice.iloc[-1].name.strftime('%Y-%m-%d')}, {currPrice}')
        df_filterStk.loc[df_filterStk['Code'] == stkCode, 'BF_HALF_YEAR_DATE'] = bfDate
        df_filterStk.loc[df_filterStk['Code'] == stkCode, 'BF_HALF_YEAR_CLOSE'] = bfPrice
        df_filterStk.loc[df_filterStk['Code'] == stkCode, 'HALF_YEAR_RATIO'] = (currPrice - bfPrice) / bfPrice * 100 
    except Exception as e:
        print(f'{e.args}')


filtered_df = df_filterStk[df_filterStk['HALF_YEAR_RATIO'] >= 10]
filtered_df